# Artificial Neural Networks

Martin Kersner

Inspired by http://neuralnetworksanddeeplearning.com/ and https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/src/network.py.

* stochastic gradient descent learning
* backpropagation
* forward pass
* backward pass

TODO Perceptron model explanation

In [37]:
import random
import numpy as np
from utils import *

TODO Image of network

In [21]:
# TODO MATRIX MULTIPLICATION EXAMPLE

In [7]:
# DEFINE AND INITIALIZE

# The first layer containing 2 neurons, the second layer 3 neurons, and the third layer 1 neuron.  
layers = [2, 3, 1]

# Altogether 3 layers.
num_layers = len(layers)

# The biases and weights for the network are initialized randomly, using a Gaussian distribution
# with mean 0, and variance 1. The first layer is assumed to be an input layer, and by convention we
# won't set any biases for those neurons.
biases  = [np.random.randn(y, 1) for y in layers[1:]]
weights = [np.random.randn(y, x) for x, y in zip(layers[:-1], layers[1:])]

In [27]:
# BIASES in layers
for b in biases:
    print(len(b))

3
1


In [16]:
# BIAS VALUES
print(biases)

[array([[ 0.03719562],
       [-0.42240308],
       [ 2.06990647]]), array([[ 1.60830567]])]


In [30]:
# WEIGHTS in layers
for w in weights:
    print(w.shape)

(3, 2)
(1, 3)


In [31]:
# WEIGHTS
print(weights)

[array([[-0.40543241,  0.31217431],
       [ 1.03464177,  0.84255726],
       [-0.39655793, -0.66093914]]), array([[-0.29819302, -0.49079507,  0.54251342]])]


## Sigmoid activation function
* introduces non-linearity

<img src="files/sigmoid.png">

In [35]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

In [34]:
def feedforward(biases, weights, a):
    for b, w in zip(biases, weights):
        a = sigmoid(np.dot(w, a)+b)
    return a

### Backpropagation

### Derivation of sigmoid

In [44]:
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))
        
def backprop(biases, weights, x, y):
    """Return a tuple ``(nabla_b, nabla_w)`` representing the
    gradient for the cost function C_x.  ``nabla_b`` and
    ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
    to ``self.biases`` and ``self.weights``."""
    
    nabla_b = zeros_like(biases)
    nabla_w = zeros_like(weights)
    
    # feedforward
    activation = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    
    for b, w in zip(biases, weights):
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
        
    # backward pass
    delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())
    # Note that the variable l in the loop below is used a little
    # differently to the notation in Chapter 2 of the book.  Here,
    # l = 1 means the last layer of neurons, l = 2 is the
    # second-last layer, and so on.  It's a renumbering of the
    # scheme in the book, used here to take advantage of the fact
    # that Python can use negative indices in lists.
    for l in xrange(2, self.num_layers):
        z = zs[-l]
        sp = sigmoid_prime(z)
        delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
        nabla_b[-l] = delta
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
    return (nabla_b, nabla_w)

## Minibatch
* TODO minibatch explanation
* TOOD update update explanation

In [41]:
# eta - learning rate
def update_mini_batch(biases, weights, mini_batch, eta):
    def update(v, nv, eta, mini_batch_size):
        return v-(eta/mini_batch_size)*nv
    
    def cumulate(nabla_v, delta_nabla_v):
        return [nv+dnv for nv, dnv in zip(nabla_v, delta_nabla_v)]
        
    nabla_b = zeros_like(biases)
    nabla_w = zeros_like(weights)
    
    # !! TODO explanation
    for x, y in mini_batch:
        delta_nabla_b, delta_nabla_w = backprop(biases, weights, x, y)
        nabla_b = cumulate(nabla_b, delta_nabla_b)
        nabla_w = cumulate(nabla_w, delta_nabla_w)
    
    mini_batch_size = len(mini_batch)
    
    weights = [update(w, nw, eta, mini_batch_size)
               for w, nw in zip(weights, nabla_w)]
    
    biases = [update(b, nb, eta, mini_batch_size)
              for b, nb in zip(biases, nabla_b)]
    
    return biases, weights

* Stochastic gradient descent
* Epoch

In [43]:
def SGD(biases, weights, training_data, epochs, mini_batch_size, eta, test_data=None):
    """Train the neural network using mini-batch stochastic
    gradient descent.  The ``training_data`` is a list of tuples
    ``(x, y)`` representing the training inputs and the desired
    outputs.  The other non-optional parameters are
    self-explanatory.  If ``test_data`` is provided then the
    network will be evaluated against the test data after each
    epoch, and partial progress printed out.  This is useful for
    tracking progress, but slows things down substantially."""
    
    if test_data:
        n_test = len(test_data)
        
    n = len(training_data)
    
    for j in xrange(epochs):
        random.shuffle(training_data)
        
        mini_batches = [training_data[k:k+mini_batch_size] for k in xrange(0, n, mini_batch_size)]
        
        for mini_batch in mini_batches:
            biases, weights = update_mini_batch(biases, weights, mini_batch, eta)
            
        if test_data:
            print("Epoch {0}: {1} / {2}".format(j, evaluate(test_data), n_test))
        else:
            print("Epoch {0} complete".format(j))
            
    return biases, weights

* derivation of sigmoid

In [ ]:
def evaluate(self, test_data):
    """Return the number of test inputs for which the neural
    network outputs the correct result. Note that the neural
    network's output is assumed to be the index of whichever
    neuron in the final layer has the highest activation."""
    test_results = [(np.argmax(self.feedforward(x)), y)
                    for (x, y) in test_data]
    return sum(int(x == y) for (x, y) in test_results)

def cost_derivative(self, output_activations, y):
    """Return the vector of partial derivatives \partial C_x /
    \partial a for the output activations."""
    return (output_activations-y)

## Example usage

## Keras